<a href="https://colab.research.google.com/github/harnalashok/deeplearning/blob/main/keras_hyperparameterOpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html


In [1]:
!pip install -q -U keras-tuner

In [2]:
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt



In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [4]:

# A shortname for kears fashion_mnist module
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
# What all this module contains
dir(fashion_mnist)  

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'load_data']

In [7]:
help(fashion_mnist.load_data)

Help on function load_data in module tensorflow.python.keras.datasets.fashion_mnist:

load_data()
    Loads the Fashion-MNIST dataset.
    
    This is a dataset of 60,000 28x28 grayscale images of 10 fashion categories,
    along with a test set of 10,000 images. This dataset can be used as
    a drop-in replacement for MNIST. The class labels are:
    
    | Label | Description |
    |:-----:|-------------|
    |   0   | T-shirt/top |
    |   1   | Trouser     |
    |   2   | Pullover    |
    |   3   | Dress       |
    |   4   | Coat        |
    |   5   | Sandal      |
    |   6   | Shirt       |
    |   7   | Sneaker     |
    |   8   | Bag         |
    |   9   | Ankle boot  |
    
    Returns:
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    
        **x_train, x_test**: uint8 arrays of grayscale image data with shape
          (num_samples, 28, 28).
    
        **y_train, y_test**: uint8 arrays of labels (integers in range 0-9)
          with shape (

In [6]:
# Data is downloaded. Download occurs only once.
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [159]:
# Where is data stored. Let us examine our Virtual Disk
!ls /

bin	 datalab  home	 lib64	opt   run   swift	       tmp    var
boot	 dev	  lib	 media	proc  sbin  sys		       tools
content  etc	  lib32  mnt	root  srv   tensorflow-1.15.2  usr


In [160]:
ls -la /root/.keras/datasets


total 12
drwxr-xr-x 3 root root 4096 Jan 10 07:31 ./
drwxr-xr-x 3 root root 4096 Jan 10 07:31 ../
drwxr-xr-x 2 root root 4096 Jan 10 07:31 fashion-mnist/


In [161]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((60000, 28, 28), (10000, 28, 28), (60000,), (10000,))

In [7]:
# Observe some pixel intensity values
x_train[10,10,:10]
y_train

array([  0,   0,   0,   0,   0,  31, 174,  28, 126, 153], dtype=uint8)

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [8]:
x_train = x_train/255
x_test = x_test/255
x_train[10,10,:10]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.12156863, 0.68235294, 0.10980392, 0.49411765, 0.6       ])

In [218]:
items = [ "T-shirt/top","Trouser", "Pullover", "Dress" , "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot" ]

In [165]:
items[y_train[0]]

'Ankle boot'

In [166]:
model = keras.models.Sequential(
                         [keras.layers.Flatten(input_shape = (28,28)),
                          keras.layers.Dense(100,activation = 'relu'),
                          keras.layers.Dense(100,activation = 'relu'),
                          keras.layers.Dense(10, activation = "softmax")]
    

)

In [167]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


In [168]:
model.layers

In [169]:
hidden1_layer = model.layers[1]
hidden1_layer.name

'dense_3'

In [170]:
weights,biases = hidden1_layer.get_weights()

In [171]:
weights
weights.shape    # (784, 100)

array([[-0.03217274, -0.02752706,  0.06193034, ..., -0.01452731,
         0.07155424, -0.05403588],
       [ 0.07254662,  0.01285969,  0.06116387, ..., -0.04698194,
         0.00498222, -0.01793279],
       [ 0.05911782, -0.04220738,  0.01135968, ...,  0.07136384,
         0.07098435,  0.01250432],
       ...,
       [ 0.03572796, -0.0286063 , -0.07818979, ..., -0.00023584,
        -0.03797683, -0.03111191],
       [ 0.05817227,  0.03888528, -0.02108637, ...,  0.01769986,
        -0.07005081,  0.04509826],
       [-0.00802279, -0.03365347, -0.07014278, ..., -0.05116423,
         0.04485649,  0.03364868]], dtype=float32)

(784, 100)

In [172]:
biases
biases.shape    # (100,)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

(100,)

In [219]:
model.compile(loss = 'sparse_categorical_crossentropy',  metrics = ['accuracy'] )

In [174]:
model.fit(x_train,y_train,  epochs = 10,verbose = 1)

Epoch 1/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.6406 - accuracy: 0.7698
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3844 - accuracy: 0.8597
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3505 - accuracy: 0.8729
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3468 - accuracy: 0.8775
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3409 - accuracy: 0.8824
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3344 - accuracy: 0.8850
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3339 - accuracy: 0.8848
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3287 - accuracy: 0.8885
Epoch 9/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3244 - accuracy: 0.8904
Epoch 10/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.318

In [12]:
 def build_model1(hp):
   n_size1 = hp.Int('n_size1', 32, 200, default=64)
   n_size2 = hp.Int('n_size2', 32, 200, default=64)
   dropout = hp.Float('dropout', 0, 0.5, step=0.1, default=0.5)
   #activation = hp.Choice('activation' , ['relu', 'tanh'])
   model = keras.models.Sequential(
                                    [keras.layers.Flatten(input_shape = [28,28]),
                                     keras.layers.Dense(n_size1,activation = 'relu'),
                                     keras.layers.Dense(n_size2,activation = 'relu'),
                                     keras.layers.Dense(10, activation = "softmax")
                                    ]
                                  )
   model.compile(
                 loss = 'sparse_categorical_crossentropy',
                 metrics = ['accuracy'])
   return model

In [13]:
tuner = kt.Hyperband(
                     build_model1,
                     objective='val_accuracy',
                     max_epochs=30,
                     hyperband_iterations=2
                     )

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [ ]:
tuner.search(x_train,y_train,
             validation_data=(x_test,y_test),
             epochs=30,
             callbacks=[
                        tf.keras.callbacks.EarlyStopping(patience=1)
                        ] 
             )

Trial 34 Complete [00h 00m 13s]
val_accuracy: 0.8575999736785889

Best val_accuracy So Far: 0.8658000230789185
Total elapsed time: 00h 06m 41s

Search: Running Trial #35

Hyperparameter    |Value             |Best Value So Far 
n_size1           |130               |65                
n_size2           |57                |92                
dropout           |0.5               |0.2               
activation        |relu              |relu              
tuner/epochs      |4                 |2                 
tuner/initial_e...|2                 |0                 
tuner/bracket     |3                 |3                 
tuner/round       |1                 |0                 
tuner/trial_id    |9307b80e4a4160e...|None              

Epoch 3/4
1875/1875 [==============================] - 6s 3ms/step - loss: 0.6378 - accuracy: 0.7731 - val_loss: 0.4190 - val_accuracy: 0.8451
Epoch 4/4
1081/1875 [================>.............] - ETA: 2s - loss: 0.3910 - accuracy: 0.8590